# Load decoding results
beacuse we only want to compute tfr for the specific channels

In [1]:
# import functions
import sys
import os
from pathlib import Path
import pandas as pd
import re
import numpy as np  
import submitit
import os 
import mne
sys.path.append(str(Path(os.getcwd()).resolve().parent)) # add the root of the project to the python path
from extraction_data import get_pid_eid_pairs
from config import paths
from functions import get_TFR, load_decoding_results, selective_channels , preprocess_LFP, split_trials


# get the pid and eid of the sessions to analyze
regionOfInterest = ['VISp', 'VISpm', 'VISam', 'VISa', 'VISrl', 'VISal', 'VISli', 'VISl'] # regions to include
pid_eid_pairs = get_pid_eid_pairs( regions = regionOfInterest,only_passive= True)

# get the decoding results
decoding_results_dir = os.path.join(str(Path(os.getcwd()).resolve().parent), 'Decoding_spikes', 'results')
results = load_decoding_results(pid_eid_pairs, suffix = 'v1_TPassive_TActive' , dir = decoding_results_dir)
right_selective, left_selective, right_sensitive, left_sensitive, neutral = selective_channels(results)
print('Right selective: ', len(right_selective))
print('Left selective: ', len(left_selective))
RS_ch = right_selective['acronyms']
LS_ch = left_selective['acronyms']

# print the number of selective channels per region
data = []
for region in regionOfInterest:
    RS_count = len([ch for ch in RS_ch if re.match(rf'^{region}[12456]', ch)])
    LS_count = len([ch for ch in LS_ch if re.match(rf'^{region}[12456]', ch)])
    data.append([region, RS_count, LS_count])

# Create a DataFrame
df = pd.DataFrame(data, columns=['Region', 'Right Selective', 'Left Selective'])
print(df)

Connected to https://openalyx.internationalbrainlab.org as user "intbrainlab"
Total number of possible probe insertions: 205
Number of sessions without passive data: 76
Number of sessions with passive data: 129
Number of missing pids: 121
 number of total channnels 272
Right selective:  1
Left selective:  6
  Region  Right Selective  Left Selective
0   VISp                1               6
1  VISpm                0               0
2  VISam                0               0
3   VISa                0               0
4  VISrl                0               0
5  VISal                0               0
6  VISli                0               0
7   VISl                0               0


# define parameters 

## Preprocessing Parameters
-   **`tmin`**: Start time of the epoch in seconds. Default is -0.5.
-   **`tmax`**:  End time of the epoch in seconds. Default is 1.
-   **`cpus`**:  Number of CPUs to use for filtering. Default is 1.
-   **`bandpass_filter`**: List containing low and high frequency for bandpass filter. Default is [None, None].
-   **`csd`**: Whether to compute Current Source Density (CSD). Default is False.
-   **`bipolar`** Whether to compute bipolar signals. Default is False.



## split data into two parts

#### 1 BiasRight_BiasLeft_anticip

- TFR for left  - right  during anticiaption of the target in left selective electrodes

- TFR  right  - left  during anticipation of the target in right selective electrodes
  
```python
condition1_trial = np.where(meta['probabilityLeft'] == 0.2 )[0]
condition2_trial = np.where(meta['probabilityLeft'] == 0.8 )[0]
```

#### 2 BiasLeft_BiasRight_stimLeft

- TFR left stimulus present 80 - TFR left stimulus present 20 in left selective electrodes

```python
condition1_trial = np.where((meta['probabilityLeft'] == 0.8) & (meta['contrastLeft'] > 0.2))[0]
condition2_trial = np.where((meta['probabilityLeft'] == 0.2) & (meta['contrastLeft'] > 0.2))[0]
```
#### 3 BiasLeft_BiasRight_stimRight
- TFR right stimulus present 80 - TFR right stimulus present 20 in right selective electrodes
  
```python
condition1_trial = np.where((meta['probabilityLeft'] == 0.8) & (meta['contrastRight'] > 0.2))[0]
condition2_trial = np.where((meta['probabilityLeft'] == 0.2) & (meta['contrastRight'] > 0.2))[0]
```


#### 4 BiasLeft_BiasRight_NoStimLeft
- TFR left stimulus absent 80 - TFR left stimulus absent 20 in left selective electrodes
```python
condition1_trial = np.where((meta['probabilityLeft'] == 0.8) & (~(meta['contrastLeft'] > 0)))[0]
condition2_trial = np.where((meta['probabilityLeft'] == 0.2) & (~(meta['contrastLeft'] > 0)))[0]
```
#### 5 BiasLeft_BiasRight_NoStimRight
- TFR right stimulus absent 80 - TFR right stimulus absent 20 in right selective electrodes
```python
condition1_trial = np.where((meta['probabilityLeft'] == 0.8) & (~(meta['contrastRight'] > 0)))[0]
condition2_trial = np.where((meta['probabilityLeft'] == 0.2) & (~(meta['contrastRight'] > 0)))[0]
```

In [2]:

cpus = 5 # for TFR and also bandpass filter
Parameters_preprocessing = { 'tmin': -0.5, 'tmax': 1, 'cpus': cpus, 'bandpass_filter': [None, None],  'csd': False, 'bipolar': True}


# parameters two split trials
remove_first_trials_of_block = True 
min_trial = 0
# 1  TFR for different conditions (see the conditions detail  in the block below)
conditions = 'BiasLeft_BiasRight_NoStimLeft'  
selectives = left_selective # pid and channels to include
#2
# conditions = 'BiasLeft_BiasRight_NoStimRight'  
# selectives = right_selective
# 3
# conditions =  'BiasLeft_BiasRight_stimRight'
# selectives = right_selective
# 4
# conditions = 'BiasLeft_BiasRight_stimLeft'
# selectives = left_selective
# 5
# conditions = 'BiasRight_BiasLeft_anticip' 
# tmin = -0.7
# tmax = 0.1
# selectives = pd.concat([right_selective, left_selective])

# TFR parameters
freqs = np.concatenate([np.arange(1, 10, 1), np.arange(10, 45, 1)])
n_cycles = freqs / 2. # increase the denominator to increase the temporal resolution
time_bandwidth = 3.5 # lower the time-bandwidth parameter, which reduces the time-length of the tapers to increases temporal resolution
overwrite = False




    

# compute TFR

In [3]:



def submit_get_TFR(pid, preprocess_params = Parameters_preprocessing,  freqs = freqs, n_cycles = n_cycles, time_bandwidth = time_bandwidth, condition = conditions, selectives = selectives, remove_first_trials_of_block = remove_first_trials_of_block, min_trial = min_trial, overwrite = overwrite, cpus = cpus):
    ch_index = selectives.loc[selectives['pid'] == pid, 'ch_indexs'].tolist()
    eid = [eid for p, eid in pid_eid_pairs if p == pid][0]
    if os.path.exists(f'results/{pid}_{condition}.npy') and not overwrite:
        print(f'{pid}_{condition} already done')
        return

    # preprocess the LFP
    epochs_mne = preprocess_LFP(pid, eid,ch_index,  **preprocess_params)

    # split trials
    epochs_1, epochs_2 = split_trials(epochs_mne, condition , min_trial , remove_first_trials_of_block )

    # TFR
    averageTFR_1 = mne.time_frequency.tfr_multitaper(epochs_1, freqs=freqs, n_cycles=n_cycles, time_bandwidth=time_bandwidth, return_itc=False, average=True, n_jobs= cpus)
    averageTFR_2 = mne.time_frequency.tfr_multitaper(epochs_2, freqs=freqs, n_cycles=n_cycles, time_bandwidth=time_bandwidth, return_itc=False, average=True, n_jobs= cpus)
    
    pid_TFR = {'TFR_1': averageTFR_1, 'TFR_2': averageTFR_2}
    # save the TFR in 
    save_path = os.path.join('results', f'{pid}_{condition}.npy')
    np.save(save_path, pid_TFR)
    return 



# remove the first and last channel because of CSD
if Parameters_preprocessing['csd'] == True:
    selectives = selectives[~selectives['ch_indexs'].isin([0, 383])]

# get the pids that are not already done
pids = np.unique(selectives['pid'].values)

# parallel computing

In [ ]:
# submit jobs
executor = submitit.AutoExecutor(folder='logs')
executor.update_parameters(slurm_array_parallelism= 30 , mem_gb=20, timeout_min= 120, slurm_partition= 'CPU',  cpus_per_task= cpus)
# Submit all jobs using map_array                                                                                              
all_jobs = executor.map_array(submit_get_TFR, pids)
print('Number of pids submited: ', len(all_jobs))

# local machine

In [5]:
for pid in pids:
    submit_get_TFR(pid)